# Data Augmentation Analysis for Bruise Detection

This notebook compares different data augmentation strategies:
1. Baseline (no augmentation)
2. Horizontal flip only
3. Rotation only (±20 degrees)
4. Zoom only (±20%)
5. All augmentations combined

We'll use the same model architecture and preprocessing from the main CNN notebook.

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models
import os
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
import seaborn as sns
import pandas as pd

In [5]:
def load_binary_data(data_dir='dataset/Wound_dataset copy', img_size=(224, 224)):
    images = []
    labels = []
    
    # Check if directory exists
    if not os.path.exists(data_dir):
        raise ValueError(f"Dataset directory not found: {data_dir}")
    
    # Process bruise images (positive class)
    bruise_path = os.path.join(data_dir, 'Bruises')
    if not os.path.exists(bruise_path):
        raise ValueError(f"Bruises directory not found: {bruise_path}")
    
    for img_name in os.listdir(bruise_path):
        if img_name.lower().endswith(('.jpg', '.jpeg', '.png')):
            img_path = os.path.join(bruise_path, img_name)
            try:
                img = Image.open(img_path)
                img = img.convert('RGB')
                img = img.resize(img_size)
                img_array = np.array(img) / 255.0
                
                images.append(img_array)
                labels.append(1)  # 1 for bruise
            except Exception as e:
                print(f"Error loading {img_path}: {e}")
    
    # Process normal skin images (negative class)
    normal_path = os.path.join(data_dir, 'Normal')
    if not os.path.exists(normal_path):
        raise ValueError(f"Normal directory not found: {normal_path}")
    
    for img_name in os.listdir(normal_path):
        if img_name.lower().endswith(('.jpg', '.jpeg', '.png')):
            img_path = os.path.join(normal_path, img_name)
            try:
                img = Image.open(img_path)
                img = img.convert('RGB')
                img = img.resize(img_size)
                img_array = np.array(img) / 255.0
                
                images.append(img_array)
                labels.append(0)  # 0 for normal
            except Exception as e:
                print(f"Error loading {img_path}: {e}")
    
    if not images:
        raise ValueError("No images were loaded. Check the dataset structure and file formats.")
    
    return np.array(images), np.array(labels)

# Try loading the dataset with explicit error handling
try:
    print("Attempting to load dataset...")
    print("Current working directory:", os.getcwd())
    X, y = load_binary_data()
    print("Dataset shape:", X.shape)
    print("Number of bruise images:", np.sum(y == 1))
    print("Number of normal images:", np.sum(y == 0))
except Exception as e:
    print(f"Error loading dataset: {e}")
    print("Please verify the dataset path and structure.")

Attempting to load dataset...
Current working directory: C:\Users\Rodney Lei Estrada\xcode\bruise_detection_project
Dataset shape: (442, 224, 224, 3)
Number of bruise images: 242
Number of normal images: 200


## Model Architecture and Training Functions

We'll use the same CNN architecture as the main notebook, but train it with different augmentation strategies.

In [6]:
def create_model(input_shape):
    model = models.Sequential([
        # Input layer
        layers.Input(shape=input_shape),
        
        # Simple CNN architecture
        layers.Conv2D(32, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        
        # Flatten layer
        layers.Flatten(),
        
        # Dense layer
        layers.Dense(64, activation='relu'),
        
        # Output layer (binary classification)
        layers.Dense(1, activation='sigmoid')
    ])
    
    return model

def train_model_with_augmentation(X_train, y_train, X_val, y_val, augmentation_config=None, model_name="baseline"):
    # Create data generator with augmentation
    if augmentation_config is not None:
        train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
            **augmentation_config
        )
    else:
        train_datagen = tf.keras.preprocessing.image.ImageDataGenerator()
    
    # Create and compile model
    input_shape = X_train[0].shape
    model = create_model(input_shape)
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy', 'Precision', 'Recall']
    )
    
    # Train the model
    history = model.fit(
        train_datagen.flow(X_train, y_train, batch_size=32),
        epochs=10,
        validation_data=(X_val, y_val),
        callbacks=[
            tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)
        ]
    )
    
    return model, history

# Define augmentation configurations
augmentation_configs = {
    'baseline': None,
    'flip': {'horizontal_flip': True},
    'rotation': {'rotation_range': 20},
    'zoom': {'zoom_range': 0.2},
    'all': {
        'horizontal_flip': True,
        'rotation_range': 20,
        'zoom_range': 0.2
    }
}

# Split data only if X and y are not empty
if X.size == 0 or y.size == 0:
    print("Warning: X and/or y are empty. Please check your dataset loading step.")
    X_train = X_val = X_test = y_train = y_val = y_test = np.array([])
else:
    # First split off test set (10%)
    X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

    # Split remaining 90% into train (70%) and validation (20%)
    X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.20, random_state=42)

    print("\nData split sizes:")
    print(f"Training set: {len(X_train)} images ({len(X_train)/len(X)*100:.1f}%)")
    print(f"Validation set: {len(X_val)} images ({len(X_val)/len(X)*100:.1f}%)")
    print(f"Test set: {len(X_test)} images ({len(X_test)/len(X)*100:.1f}%)")

NameError: name 'X' is not defined

## Train Models with Different Augmentation Strategies

We'll train 5 different models and compare their performance:

In [ ]:
# Dictionary to store results
results = {}

# Train models with different augmentation strategies
for aug_name, aug_config in augmentation_configs.items():
    print(f"\nTraining model with {aug_name} augmentation...")
    model, history = train_model_with_augmentation(
        X_train, y_train,
        X_val, y_val,
        augmentation_config=aug_config,
        model_name=aug_name
    )
    
    # Evaluate on test set
    test_loss, test_accuracy, test_precision, test_recall = model.evaluate(X_test, y_test)
    
    # Get predictions for ROC curve
    y_pred_prob = model.predict(X_test)
    y_pred = (y_pred_prob > 0.5).astype(int)
    
    # Calculate ROC curve and AUC
    fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
    roc_auc = auc(fpr, tpr)
    
    # Store results
    results[aug_name] = {
        'history': history.history,
        'test_accuracy': test_accuracy,
        'test_precision': test_precision,
        'test_recall': test_recall,
        'confusion_matrix': confusion_matrix(y_test, y_pred),
        'roc_auc': roc_auc,
        'fpr': fpr,
        'tpr': tpr
    }
    
    print(f"\nTest Results for {aug_name}:")
    print(f"Accuracy: {test_accuracy:.4f}")
    print(f"Precision: {test_precision:.4f}")
    print(f"Recall: {test_recall:.4f}")
    print(f"ROC AUC: {roc_auc:.4f}")
    print("\nConfusion Matrix:")
    print(confusion_matrix(y_test, y_pred))

## Visualization and Analysis

In [ ]:
def plot_training_histories(results):
    metrics = ['accuracy', 'loss', 'precision', 'recall']
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    for idx, metric in enumerate(metrics):
        row = idx // 2
        col = idx % 2
        
        for aug_name in results.keys():
            history = results[aug_name]['history']
            axes[row, col].plot(history[metric], label=f'{aug_name}')
            axes[row, col].plot(history[f'val_{metric}'], label=f'{aug_name} (val)', linestyle='--')
        
        axes[row, col].set_title(f'Model {metric.capitalize()}')
        axes[row, col].set_xlabel('Epoch')
        axes[row, col].set_ylabel(metric.capitalize())
        axes[row, col].legend()
        axes[row, col].grid(True)
    
    plt.tight_layout()
    plt.show()

def plot_roc_curves(results):
    plt.figure(figsize=(10, 8))
    
    for aug_name in results.keys():
        plt.plot(results[aug_name]['fpr'], 
                results[aug_name]['tpr'],
                label=f'{aug_name} (AUC = {results[aug_name]["roc_auc"]:.3f})')
    
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curves for Different Augmentation Strategies')
    plt.legend(loc="lower right")
    plt.grid(True)
    plt.show()

def plot_performance_comparison(results):
    metrics = ['test_accuracy', 'test_precision', 'test_recall', 'roc_auc']
    x = np.arange(len(results))
    width = 0.2
    
    fig, ax = plt.subplots(figsize=(12, 6))
    
    for i, metric in enumerate(metrics):
        values = [results[aug_name][metric] for aug_name in results.keys()]
        ax.bar(x + i*width, values, width, label=metric.replace('test_', '').capitalize())
    
    ax.set_ylabel('Score')
    ax.set_title('Performance Comparison Across Augmentation Strategies')
    ax.set_xticks(x + width * (len(metrics)-1)/2)
    ax.set_xticklabels(list(results.keys()))
    ax.legend()
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

# Plot results
plot_training_histories(results)
plot_roc_curves(results)
plot_performance_comparison(results)

# Print summary table
summary_data = {
    'Augmentation': [],
    'Accuracy': [],
    'Precision': [],
    'Recall': [],
    'ROC AUC': []
}

for aug_name, result in results.items():
    summary_data['Augmentation'].append(aug_name)
    summary_data['Accuracy'].append(f"{result['test_accuracy']:.4f}")
    summary_data['Precision'].append(f"{result['test_precision']:.4f}")
    summary_data['Recall'].append(f"{result['test_recall']:.4f}")
    summary_data['ROC AUC'].append(f"{result['roc_auc']:.4f}")

summary_df = pd.DataFrame(summary_data)
print("\nPerformance Summary:")
print(summary_df.to_string(index=False))

## Analysis and Insights

From the results above, we can analyze:

1. Overall Performance:
   - Compare accuracy, precision, and recall across different augmentation strategies
   - Look at ROC AUC scores to assess overall discriminative ability

2. Training Dynamics:
   - How different augmentations affect learning speed
   - Whether certain augmentations help prevent overfitting

3. Best Augmentation Strategy:
   - Which augmentation or combination worked best
   - Trade-offs between different metrics

4. Recommendations:
   - Best augmentation strategy for this specific task
   - Potential improvements or modifications

In [ ]:
# Debug: Print available paths and verify dataset location
import os

# Define possible dataset paths
current_dir = os.getcwd()
absolute_dataset_path = os.path.join('C:\\Users\\Rodney Lei Estrada\\dataset\\Wound_dataset')

print(f"Current directory: {current_dir}")
print(f"Absolute dataset path: {absolute_dataset_path}")
print(f"\nChecking if dataset exists at absolute path: {os.path.exists(absolute_dataset_path)}")

if os.path.exists(absolute_dataset_path):
    print("\nContents of dataset directory:")
    print(os.listdir(absolute_dataset_path))